In [ ]:
# Nanopore pipeline to basecall and demultiple nanopore fast5 raw data

In [ ]:
# print work directory
pwd

In [ ]:
# change directory
cd /home/andhika/extrastorage/fishy/20190313_0937_MN31442_FAK11288_e5927bdc/

In [ ]:
# basecalling using guppy

~/extrastorage/SOFTWARE/ont_guppy_cpu/bin/guppy_basecaller --input_path ~/extrastorage/MinION/dum_fishy/20190313_0937_MN31442_FAK11288_e5927bdc/fast5 --save_path ~/extrastorage/MinION/dum_fishy/20190313_0937_MN31442_FAK11288_e5927bdc/basecall_2 --flowcell FLO-MIN106 --kit SQK-LSK109

In [ ]:
# combine all barcode into 1 file
cat basecall/*fastq > combined.fastq

In [ ]:
# check
grep -c '^+$' combined.fastq
# 1,800,063 million reads

In [ ]:
# demultiplex basecalled fast5 reads
qcat -f combined.fastq -b final_NPreads &> final_log

In [ ]:
# open file
less final_log 

# quit
q

In [ ]:
## EXTRACT BY SAMPLE
# create folder for analyzing by sample
mkdir dhika_files
cd dhika_files

In [ ]:
# create folder for barcode sample
mkdir sample

In [ ]:
# Tried to extract by barcode basecall to barcode
cat basecall/*.fastq | qcat -b dhika_files/sample/

In [ ]:
# create folder for blast
mkdir out_ncbi

In [ ]:
# blast
blastn -query sample/barcode01.fastq -db /extrastorage/home/refdbs/blastdb/nt/nt -num_threads 23 -out out_ncbi/barcode01_ncbi
blastn -query sample/barcode02.fastq -db /extrastorage/home/refdbs/blastdb/nt/nt -num_threads 23 -out out_ncbi/barcode02_ncbi
blastn -query sample/barcode03.fastq -db /extrastorage/home/refdbs/blastdb/nt/nt -num_threads 23 -out out_ncbi/barcode03_ncbi
blastn -query sample/barcode04.fastq -db /extrastorage/home/refdbs/blastdb/nt/nt -num_threads 23 -out out_ncbi/barcode04_ncbi
blastn -query sample/barcode05.fastq -db /extrastorage/home/refdbs/blastdb/nt/nt -num_threads 23 -out out_ncbi/barcode05_ncbi
blastn -query sample/barcode06.fastq -db /extrastorage/home/refdbs/blastdb/nt/nt -num_threads 23 -out out_ncbi/barcode06_ncbi
blastn -query sample/barcode07.fastq -db /extrastorage/home/refdbs/blastdb/nt/nt -num_threads 23 -out out_ncbi/barcode07_ncbi
blastn -query sample/barcode08.fastq -db /extrastorage/home/refdbs/blastdb/nt/nt -num_threads 23 -out out_ncbi/barcode08_ncbi
blastn -query sample/barcode09.fastq -db /extrastorage/home/refdbs/blastdb/nt/nt -num_threads 23 -out out_ncbi/barcode09_ncbi
blastn -query sample/barcode10.fastq -db /extrastorage/home/refdbs/blastdb/nt/nt -num_threads 23 -out out_ncbi/barcode10_ncbi

In [ ]:
# create folder to converst fastq to fasta
mkdir sample_fasta

In [ ]:
# convert fastq to fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode01.fastq > sample_fasta/barcode01.fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode02.fastq > sample_fasta/barcode02.fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode03.fastq > sample_fasta/barcode03.fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode04.fastq > sample_fasta/barcode04.fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode05.fastq > sample_fasta/barcode05.fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode06.fastq > sample_fasta/barcode06.fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode07.fastq > sample_fasta/barcode07.fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode08.fastq > sample_fasta/barcode08.fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode09.fastq > sample_fasta/barcode09.fasta
sed -n '1~4s/^@/>/p;2~4p' sample/barcode10.fastq > sample_fasta/barcode010.fasta

In [ ]:
# Assembling
canu -d assemble/ -p shark genomeSize=16k minReadLength=500 -nanopore-raw combined.fastq 2> canu.log
canu -d assemble/ -p barcode01 genomeSize=16k minReadLength=500 -nanopore-raw sample/barcode01.fastq 2> canu.log

In [ ]:
cp assemble/shark.contigs.fasta shark.contigs.fasta

In [ ]:
blastn -db /extrastorage/home/refdbs/blastdb/nt/nt -query sample_fasta/barcode01.fasta -out blastn/barcode01_nt.blastn -outfmt "6 qseqid sseqid pident length evalue salltitles sallacc sstart send qstart qend" -max_target_seqs 2  -max_hsps 2 -num_threads 16 -culling_limit 1 -evalue 0.001

In [ ]:
blastn -db /extrastorage/home/refdbs/blastdb/nt/nt -query sample_fasta/barcode02.fasta -out blastn/barcode02_nt.blastn -outfmt "6 qseqid sseqid pident length evalue salltitles sallacc sstart send qstart qend" -max_target_seqs 2  -max_hsps 2 -num_threads 16 -culling_limit 1 -evalue 0.001

In [ ]:
# barcode01
# 809fcd04-a1da-4889-87c6-030d3a1c2aae is the highest hits | Scyliorhinus canicula Cluster_HOXA sequence
samtools faidx sample_fasta/barcode01.fasta '809fcd04-a1da-4889-87c6-030d3a1c2aae' > samtools/809fcd04-a1da-4889-87c6-030d3a1c2aae.fasta

In [ ]:
# barcode02
#1cef951c-5c99-4862-9f3b-e170eb4ac7cd | Carcharhinus plumbeus T cell receptor gamma (TCRG) gene | FJ854492
samtools faidx sample_fasta/barcode02.fasta '1cef951c-5c99-4862-9f3b-e170eb4ac7cd' > samtools/barcode02_1cef951c-5c99-4862-9f3b-e170eb4ac7cd.fasta


In [ ]:
#Barcode 1
blastn -db /extrastorage/home/refdbs/blastdb/nt/nt -query samtools/809fcd04-a1da-4889-87c6-030d3a1c2aae.fasta -out genes_barcode1.blast -outfmt "6 qseqid sseqid pident length evalue sscinames salltitles sallacc sstart send qstart qend" -max_target_seqs 1 -num_threads 16 -culling_limit 1 -max_hsps 1 -evalue 0.001

In [ ]:
# Scyliorhinus canicula

In [ ]:
#Barcode 2
blastn -db /extrastorage/home/refdbs/blastdb/nt/nt -query samtools/barcode02_1cef951c-5c99-4862-9f3b-e170eb4ac7cd.fasta -out genes_barcode2.blast -outfmt "6 qseqid sseqid pident length evalue sscinames salltitles sallacc sstart send qstart qend" -max_target_seqs 1 -num_threads 16 -culling_limit 1 -max_hsps 1 -evalue 0.001

In [ ]:
# Carcharhinus plumbeus